In [67]:
from calculate_tissue_mask import calculate_tissue_mask
from check_if_model_parameters_changed import check_if_model_parameters_changed
from import_xml import import_xml
from save_annotation_mask import save_annotation_mask
from save_bounding_boxes import save_bounding_boxes

In [68]:
import os
import pickle
import shutil
import numpy as np
from skimage import io
import time


In [69]:
#inputs
pth = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model'
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
pthim = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x'
classcheck=0

In [70]:
#Load data
print(' ')
print('Importing annotation data...')

with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    WS = data['WS']
    umpix = data['umpix']
    cmap = data['cmap']
    classNames = data['classNames']
    nm = data['nm']
    nwhite = data['nwhite']
    
cmap2 = np.vstack(([0, 0, 0], cmap)) / 255
numclass = np.max(WS[2])
imlist = [f for f in os.listdir(pth) if f.endswith('.xml')]
numann0 = []
ctlist0 = []
outim = os.path.join(pth, 'check_annotations')
os.makedirs(outim, exist_ok=True)

 
Importing annotation data...


In [71]:
# Check that all images exist for all the .cml files contained in the folder
for imnm in imlist:
    imnm = imnm[:-4]
    tif_file = os.path.join(pthim, f'{imnm}.tif')
    jpg_file = os.path.join(pthim, f'{imnm}.jpg')
    if not os.path.isfile(tif_file) and not os.path.isfile(jpg_file):
        raise FileNotFoundError(f'Cannot find a tif or jpg file for xml file: {imnm}.xml')

In [72]:
# for each annotation file
start_time = time.time()  # Capture the start time before the loop
for idx, imnm in enumerate(imlist, start=1):
    print(f' Image {idx} of {len(imlist)}: {imnm[:-4]}')
    imnm = imnm[:-4]
    outpth = os.path.join(pth, 'data', imnm)
    annotations_file = os.path.join(outpth, 'annotations.pkl')
    
    # check if model parameters have changed
    reload_xml = check_if_model_parameters_changed(annotations_file, WS, umpix, nwhite, pthim)
    
    # skip if file hasn't been updated since last load
    if os.path.isfile(annotations_file):
        with open(annotations_file, 'rb') as f:
            data = pickle.load(f)
            dm, bb = data.get('dm', ''), data.get('bb', 0)
    else:
        dm, bb = '', 0
    
    date_modified = os.path.getmtime(os.path.join(pth, f'{imnm}.xml'))
    if dm == str(date_modified) and bb == 1 and not reload_xml:
        print(' annotation data previously loaded')
        with open(annotations_file, 'rb') as f:
            data = pickle.load(f)
            numann, ctlist = data.get('numann', []), data.get('ctlist', [])
        numann0.extend(numann)
        ctlist0.extend(ctlist)
        continue
        
    if os.path.isdir(outpth):
        shutil.rmtree(outpth)
    os.makedirs(outpth)
    
    # 1 read xml annotation files and save as pkl files
    import_xml(annotations_file, os.path.join(pth, f'{imnm}.xml'), date_modified)
    print(f'annotation_file: {annotations_file}')
    print("xml_file:", os.path.join(pth, f'{imnm}.xml'))
    with open(annotations_file, 'wb') as f:
        pickle.dump({'WS': WS, 'umpix': umpix, 'nwhite': nwhite, 'pthim': pthim}, f)
        
    # 2 fill annotation outlines and delete unwanted pixels
    I0, TA, _ = calculate_tissue_mask(pthim, imnm)
    J0 = save_annotation_mask(I0, outpth, WS, umpix, TA, 1)
    io.imsave(os.path.join(outpth, 'view_annotations.tif'), J0.astype(np.uint8))
    
    # show mask in color
    J2 = J0 + 1
    cmap3 = cmap2[np.min(J2):np.max(J2) + 1, :]
    mask = cmap3[J2]
    I = I0[::2, ::2, :].astype(np.float64) * 0.5 + mask[::2, ::2, :] * 0.5
    io.imsave(os.path.join(outim, f'{imnm}.jpg'), (I * 255).astype(np.uint8))
    
    # create annotation bounding boxes and update data to annotation.pkl file
    numann, ctlist = save_bounding_boxes(I0, outpth, nm, numclass)
    with open(annotations_file, 'wb') as f:
        pickle.dump({'dm': str(date_modified), 'bb': 1, 'numann': numann, 'ctlist': ctlist}, f)
    numann0.extend(numann)
    ctlist0.extend(ctlist) 
    
    print(f' Finished image in {round(time.time() - start_time)} seconds.')


 Image 1 of 2: SG_013_0061
 1. of 4. Importing annotation data from xml file
Creating file...
annotation_file: \\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\data\SG_013_0061\annotations.pkl
xml_file: \\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\SG_013_0061.xml
Existing TA loaded
     2. of 4. Interpolating annotated regions and saving mask image


KeyError: 'xyout'

In [48]:
# Load data from pickle file (example):
annotations_file = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\data\SG_013_0061\annotations.pkl'
with open(annotations_file, 'rb') as f:
    data = pickle.load(f)
xyout_df = data['xyout']
reduced_annotations = data['reduce_annotations']
dm = data['dm']
print(f'Reduced annotations: {reduced_annotations}')
print(f'Date modified: {dm}')
print('Annotation coordinates:')
print(xyout_df)

KeyError: 'xyout'